# **Notebook para selección del mejor algoritmo de Machine Learning**
Con la finalidad de encontrar el algoritmo que mas se ajusta al set de datos propuesto y con el esquema de preselección de 6 algortimos que cumplían con unas características especificas, se seleccionaron estos algoritmos para realizar las pruebas:
* Multi-Layer Perceptron (MLP)
* Random Forest (RF)
* Light Gradient Boosting Machine (LightGBM)
* Extreme Gradient Boosting (XGB)
* Support Vector Machine (SVM)
* Isolation Forest (iForest)

Se realizaron los ajustes de las librerías necesarias y se efectuó un entrenamiento asistido con los métodos ofrecidos por las librerías de cada algoritmo.

### **Estructura del notebook:**
1. Instalación de dependencias
2. Cargue de archivos CSV (gps_valid.csv)
3. Importe y carga de datos
4. Función Haversine
5. Función de generación datos spoofing
6. Entrenamiento con MLP
7. Entrenamiento con RF
8. Entrenamiento con LightGBM
9. Entrenamiento con RF con XGB
10. Entrenamiento con iForest
11. Entrenamiento con SVM



# **1. Instalación de dependencias**

In [ ]:
# === INSTALAR DEPENDENCIAS ===
!pip install --upgrade --force-reinstall numpy scipy scikit-learn pandas joblib xgboost


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 52.7 MB/s eta 0:00:00
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# **2. Cargue de archivos CSV (gps_valid.csv)**

In [ ]:
# === SUBIR CSV ===
from google.colab import files

print("🔃 Sube aquí tu archivo gps_valid.csv (datos íntegros sin spoofing)")
uploaded = files.upload()  # Selecciona gps_valid.csv


🔃 Sube aquí tu archivo gps_valid.csv (datos íntegros sin spoofing)


Saving gps_valid.csv to gps_valid.csv


# **3. Importe y carga de datos**

In [ ]:
# === IMPORTS Y CARGA DE DATOS ===
import pandas as pd
import numpy as np
import joblib
from math import radians, sin, cos, sqrt, atan2, degrees
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier
from google.colab import files as gfiles

# Nombre del CSV subido
CSV_FILE = "gps_valid.csv"

# Cargar y parsear
df = pd.read_csv(CSV_FILE, parse_dates=["UTC"])
# Separar Lat/Lon
df[["Lat","Lon"]] = df["Position"].str.split(",", expand=True).astype(float)
df = df.sort_values("UTC").reset_index(drop=True)

print(f"✅ Datos cargados: {len(df)} filas")



✅ Datos cargados: 7510 filas


# **4. Función Haversine**

In [ ]:
# === CÁLCULO DE FEATURES DE CONTINUIDAD ===
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radio terrestre en metros
    φ1, φ2 = radians(lat1), radians(lat2)
    Δφ = radians(lat2 - lat1)
    Δλ = radians(lon2 - lon1)
    a = sin(Δφ/2)**2 + cos(φ1)*cos(φ2)*sin(Δλ/2)**2
    d = 2 * R * atan2(sqrt(a), sqrt(1-a))
    y = sin(Δλ) * cos(φ2)
    x = cos(φ1)*sin(φ2) - sin(φ1)*cos(φ2)*cos(Δλ)
    θ = (degrees(atan2(y, x)) + 360) % 360
    return d, θ

def compute_continuity_features(df):
    dists, bears = [], []
    for i in range(1, len(df)):
        d, b = haversine(
            df.loc[i-1,"Lat"], df.loc[i-1,"Lon"],
            df.loc[i  ,"Lat"], df.loc[i  ,"Lon"]
        )
        dists.append(d)
        bears.append(b)
    return pd.DataFrame({
        "dist":    np.array(dists, dtype=np.float32),
        "bearing": np.array(bears, dtype=np.float32)
    })

cont_df = compute_continuity_features(df)
y_clean = np.ones(len(cont_df), dtype=int)

print(f"✅ {len(cont_df)} muestras de continuidad calculadas")


✅ 7509 muestras de continuidad calculadas


# **5. Función de generación datos spoofing**

In [ ]:
# === GENERACIÓN DE ANOMALÍAS SINTÉTICAS ===
SEED = 42
ANOMALY_RATIO = 0.2

np.random.seed(SEED)
n = len(cont_df)
n_anom = int(n * ANOMALY_RATIO)
# Seleccion de indices del dataset original
idx = np.random.choice(n, n_anom, replace=False)

# Creacion de un arreglo con valores de spoofing generados
cont_anom_features = []
for j in idx:
    # Toma de datos del dataset original
    d_orig, b_orig = cont_df.loc[j, "dist"], cont_df.loc[j, "bearing"]
    lat0, lon0 = df.loc[j, "Lat"], df.loc[j, "Lon"] # Uso de la funcion Haversine para carcualar valores de latitud y longitud

    # Generacion de parametros aleatorios fuera de ruta entre 5 y 50 kilometros (teniendo en cuenta que el radio esta en kilometros)
    jump_d = np.random.uniform(5_000, 50_000)
    jump_b = np.random.uniform(0, 360)

    # El calculo asume que se tiene un salto a una nueva posicion, simulando un spoofing
    # La anomalia estaria en el punto generado (df.loc[j])
    φ = radians(lat0)
    Δφ = (jump_d * cos(radians(jump_b))) / 6371000
    Δλ = (jump_d * sin(radians(jump_b))) / (6371000 * cos(φ))
    lat1 = lat0 + degrees(Δφ)
    lon1 = lon0 + degrees(Δλ)

    '''
    Calcular las características con la funcion  Haversine para el punto anómalo en relación con el punto anterior, asumiendo que el punto anterior es el que está en el índice j-1 en el DataFrame original (df).
    Se usara la interpretación de que el salto modifica la ubicación del punto j y que la anomalía se detecta en la característica de continuidad entre j-1 y el j modificado.
    Esto significa que el cálculo con la funcion Haversine debe realizarse entre `df.loc[j-1]` y la nueva ubicación `(lat1, lon1)`.
    Las características `dist` y `bearing` en el índice `i` dentro de `cont_df` corresponden a la continuidad entre `df.loc[i-1]` y `df.loc[i]`.
    Si queremos introducir una anomalía en el índice `j` de `cont_df`, deberíamos modificar las características de continuidad entre `df.loc[j-1]` y una nueva ubicación para `df.loc[j]`.

    Las características anómalas deberían calcularse entre `df.loc[j]` y una nueva ubicación para `df.loc[j+1]`.
    El código original usaba `df.loc[j]` como punto de inicio del salto y aplicaba el salto para obtener un nuevo punto (lat1, lon1).
    Luego calculaba Haversine entre `df.loc[j]` y este nuevo punto. Este cálculo no reemplaza directamente el valor original en `cont_df[j]` (que es entre `df[j]` y `df[j+1]`).

    Supongamos entonces que el objetivo es crear puntos de spoofing. Un paso anómalo en el índice `j` de `cont_df` significa que la transición desde `df.loc[j]` hacia `df.loc[j+1]` es anómala.
    Deberíamos tomar `df.loc[j]` como punto de partida y aplicar un salto para encontrar una nueva ubicación para el siguiente punto (que sería el punto anómalo `df.loc[j+1]`).
    '''

    lat_start, lon_start = df.loc[j, "Lat"], df.loc[j, "Lon"]

    φ_start = radians(lat_start)
    Δφ = (jump_d * cos(radians(jump_b))) / 6371000
    Δλ = (jump_d * sin(radians(jump_b))) / (6371000 * cos(φ_start))
    lat_anom_next = lat_start + degrees(Δφ)
    lon_anom_next = lon_start + degrees(Δλ)

    # Calcular las características de Haversine para este paso anómalo (entre df.loc[j] y la nueva ubicación para df.loc[j+1])
    d_anom, b_anom = haversine(lat_start, lon_start, lat_anom_next, lon_anom_next)
    cont_anom_features.append([d_anom, b_anom])

# Se convierte el listado de anomalias en un numpy array
cont_anom_features = np.array(cont_anom_features, dtype=np.float32)

y_anom = np.zeros(n_anom, dtype=int)

# Se combinan los datos del dataset entre integros 1 y anomalos o spoofing 0
X = np.vstack([cont_df.values, cont_anom_features])
y = np.concatenate([y_clean, y_anom])

print(f"✅ Dataset combinado: {X.shape[0]} muestras ({len(y_clean)} íntegros + {n_anom} anómalos)")


✅ Dataset combinado: 9010 muestras (7509 íntegros + 1501 anómalos)


# **6. Entrenamiento con MLP**

In [ ]:
# === ENTRENAMIENTO MLP ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=200,
    random_state=SEED,
    verbose=True
)
mlp.fit(X_train, y_train)
print("✅ MLP entrenado correctamente")


Iteration 1, loss = 5.57524007
Iteration 2, loss = 2.32617669
Iteration 3, loss = 1.64645100
Iteration 4, loss = 2.05241832
Iteration 5, loss = 1.96156036
Iteration 6, loss = 2.08208372
Iteration 7, loss = 1.88712903
Iteration 8, loss = 1.53279043
Iteration 9, loss = 1.47457927
Iteration 10, loss = 1.77926970
Iteration 11, loss = 1.96757365
Iteration 12, loss = 1.93428265
Iteration 13, loss = 1.69848470
Iteration 14, loss = 1.23135989
Iteration 15, loss = 1.70129981
Iteration 16, loss = 2.41347051
Iteration 17, loss = 2.60123147
Iteration 18, loss = 1.73810149
Iteration 19, loss = 2.97231880
Iteration 20, loss = 1.59230060
Iteration 21, loss = 1.38349140
Iteration 22, loss = 2.17824599
Iteration 23, loss = 1.94167175
Iteration 24, loss = 1.87374947
Iteration 25, loss = 2.41018855
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
✅ MLP entrenado correctamente


## **6.1 Evaluación y generación modelo**

In [ ]:
# === EVALUACIÓN Y GUARDADO DEL MLP ===
y_pred = mlp.predict(X_test)
print("=== Reporte de clasificación MLP ===")
print(classification_report(y_test, y_pred, digits=4))
print("ROC AUC (MLP):", roc_auc_score(y_test, mlp.predict_proba(X_test)[:,1]))

# Serializar y descargar el modelo
joblib.dump(mlp, "mlp_continuity_model.pkl")
print("✅ MLP serializado en 'mlp_continuity_model.pkl'")
gfiles.download("mlp_continuity_model.pkl")


=== Reporte de clasificación MLP ===
              precision    recall  f1-score   support

           0     0.6489    0.8133    0.7219       300
           1     0.9607    0.9121    0.9358      1502

    accuracy                         0.8957      1802
   macro avg     0.8048    0.8627    0.8288      1802
weighted avg     0.9088    0.8957    0.9002      1802

ROC AUC (MLP): 0.9358710608078118
✅ MLP serializado en 'mlp_continuity_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **7. Entrenamiento con RF**

In [ ]:
# === ENTRENAMIENTO DEL RANDOM FOREST ===
from sklearn.ensemble import RandomForestClassifier

# División estratificada (usa X, y ya definidos en celdas anteriores)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Configuración y entrenamiento del Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=SEED,
    n_jobs=-1
)
rf.fit(X_train, y_train)
print("✅ Random Forest entrenado correctamente")


✅ Random Forest entrenado correctamente


## **7.1 Evaluación y generación modelo**

In [ ]:
# === EVALUACIÓN Y GUARDADO DEL RANDOM FOREST ===
# Predicciones y métricas
y_pred = rf.predict(X_test)
print("=== Reporte de clasificación RF ===")
print(classification_report(y_test, y_pred, digits=4))
print("ROC AUC (RF):", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

# Serializar y descargar el modelo
joblib.dump(rf, "rf_continuity_model.pkl")
print("✅ RF serializado en 'rf_continuity_model.pkl'")
gfiles.download("rf_continuity_model.pkl")


=== Reporte de clasificación RF ===
              precision    recall  f1-score   support

           0     0.9601    0.9633    0.9617       300
           1     0.9927    0.9920    0.9923      1502

    accuracy                         0.9872      1802
   macro avg     0.9764    0.9777    0.9770      1802
weighted avg     0.9873    0.9872    0.9872      1802

ROC AUC (RF): 0.9963371060807812
✅ RF serializado en 'rf_continuity_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **8. Entrenamiento con LightGBM**

In [ ]:
# === ENTRENAR con LightGBM ===
!pip install lightgbm

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

# Usamos X, y, SEED ya definidos en las celdas anteriores
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Configuración y entrenamiento del modelo LightGBM
lgbm = LGBMClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    random_state=SEED,
    n_jobs=-1
)
lgbm.fit(X_train, y_train)
print("✅ LightGBM entrenado correctamente")


[LightGBM] [Info] Number of positive: 6007, number of negative: 1201
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000783 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7208, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.833380 -> initscore=1.609771
[LightGBM] [Info] Start training from score 1.609771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


## **8.1 Evaluación y generación modelo**

In [ ]:
# === EVALUACIÓN Y GUARDADO LightGBM ===
from sklearn.metrics import classification_report, roc_auc_score
import joblib
from google.colab import files as gfiles

# Predicción y métricas
y_pred = lgbm.predict(X_test)
print("=== Reporte de clasificación LightGBM ===")
print(classification_report(y_test, y_pred, digits=4))
print("ROC AUC (LightGBM):", roc_auc_score(y_test, lgbm.predict_proba(X_test)[:,1]))

# Serializar y descargar el modelo
joblib.dump(lgbm, "lgbm_continuity_model.pkl")
print("✅ LightGBM serializado en 'lgbm_continuity_model.pkl'")

gfiles.download("lgbm_continuity_model.pkl")


=== Reporte de clasificación LightGBM ===
              precision    recall  f1-score   support

           0     0.9689    0.9333    0.9508       300
           1     0.9868    0.9940    0.9904      1502

    accuracy                         0.9839      1802
   macro avg     0.9778    0.9637    0.9706      1802
weighted avg     0.9838    0.9839    0.9838      1802

ROC AUC (LightGBM): 0.9982789613848203
✅ LightGBM serializado en 'lgbm_continuity_model.pkl'


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **9. Entrenamiento con RF con XGB**

In [ ]:
# === ENTRENAMIENTO DE RF & XGB ===
!pip install xgboost

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# División estratificada (X, y, SEED ya definidos en celdas anteriores)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# 1) RandomForest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=SEED,
    n_jobs=-1
)
rf.fit(X_train, y_train)
print("✅ Random Forest entrenado")

# 2) XGBoost
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=SEED,
    n_jobs=-1
)
xgb.fit(X_train, y_train)
print("✅ XGBoost entrenado")


✅ Random Forest entrenado
✅ XGBoost entrenado


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [03:08:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## **9.1 Evaluación y generación modelo**

In [ ]:
# === EVALUACIÓN Y GUARDADO DE RF y XGB ===
from sklearn.metrics import classification_report, roc_auc_score
import joblib
from google.colab import files as gfiles

# -- Evaluar Random Forest --
y_pred_rf = rf.predict(X_test)
print("=== RF Classification Report ===")
print(classification_report(y_test, y_pred_rf, digits=4))
print("RF ROC AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

# -- Evaluar XGBoost --
y_pred_xgb = xgb.predict(X_test)
print("\n=== XGB Classification Report ===")
print(classification_report(y_test, y_pred_xgb, digits=4))
print("XGB ROC AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))

# -- Guardar modelos --
joblib.dump(rf, "rf_continuity_model.pkl")
joblib.dump(xgb, "xgb_continuity_model.pkl")
print("\n✅ Modelos serializados: rf_continuity_model.pkl, xgb_continuity_model.pkl")

# -- Descargar --
gfiles.download("rf_continuity_model.pkl")
gfiles.download("xgb_continuity_model.pkl")


=== RF Classification Report ===
              precision    recall  f1-score   support

           0     0.9601    0.9633    0.9617       300
           1     0.9927    0.9920    0.9923      1502

    accuracy                         0.9872      1802
   macro avg     0.9764    0.9777    0.9770      1802
weighted avg     0.9873    0.9872    0.9872      1802

RF ROC AUC: 0.9963371060807812

=== XGB Classification Report ===
              precision    recall  f1-score   support

           0     0.9660    0.9467    0.9562       300
           1     0.9894    0.9933    0.9914      1502

    accuracy                         0.9856      1802
   macro avg     0.9777    0.9700    0.9738      1802
weighted avg     0.9855    0.9856    0.9855      1802

XGB ROC AUC: 0.9984598313359965

✅ Modelos serializados: rf_continuity_model.pkl, xgb_continuity_model.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **10. Entrenamiento con iForest**

In [ ]:
# === ENTRENAMIENTO Y EVALUACIÓN DE ISOLATION FOREST ===
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score

# Entrenamos SOLO con datos íntegros (cont_df) y parametrizamos la poda de anomalías, teniendo en cuenta que este algoritmo es no supervisado
iso = IsolationForest(
    contamination=ANOMALY_RATIO,  # proporción de outliers sintéticos
    random_state=SEED
)
iso.fit(cont_df.values)
print("✅ Isolation Forest entrenado sobre datos íntegros")

# Predecimos sobre el dataset combinado X
# iso.predict:  1 = normal, -1 = anómalo
pred_iso = iso.predict(X)
y_pred = np.where(pred_iso == 1, 1, 0)

# Reporte de métricas
print("=== Reporte de clasificación IsolationForest ===")
print(classification_report(y, y_pred, digits=4))

# Para ROC AUC usamos score de anomalía (más alto = más anómalo)
scores = -iso.decision_function(X)
print("ROC AUC (IsolationForest):", roc_auc_score(y, scores))


✅ Isolation Forest entrenado sobre datos íntegros
=== Reporte de clasificación IsolationForest ===
              precision    recall  f1-score   support

           0     0.4987    0.9953    0.6644      1501
           1     0.9988    0.8000    0.8884      7509

    accuracy                         0.8325      9010
   macro avg     0.7488    0.8977    0.7764      9010
weighted avg     0.9155    0.8325    0.8511      9010

ROC AUC (IsolationForest): 0.007900534903308127


## **10.1 Evaluación y generación modelo**

In [ ]:
# === SERIALIZAR Y DESCARGAR EL MODELO ===
import joblib
from google.colab import files as gfiles

joblib.dump(iso, "iso_continuity_model.pkl")
print("✅ Isolation Forest serializado en 'iso_continuity_model.pkl'")

gfiles.download("iso_continuity_model.pkl")


✅ Isolation Forest serializado en 'iso_continuity_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **11. Entrenamiento con SVM**

In [ ]:
# === ENTRENAMIENTO DEL SVM ===
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# División estratificada (X, y y SEED ya definidos en celdas anteriores)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Configuración y entrenamiento del SVM
svm = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    probability=True,     # Para usar predict_proba()
    random_state=SEED
)
svm.fit(X_train, y_train)
print("✅ SVM entrenado correctamente")


✅ SVM entrenado correctamente


## **11.1 Evaluación y generación modelo**

In [ ]:
# === EVALUACIÓN Y GUARDADO DEL SVM ===
from sklearn.metrics import classification_report, roc_auc_score
import joblib
from google.colab import files as gfiles

# Predicciones y métricas
y_pred = svm.predict(X_test)
print("=== Reporte de clasificación SVM ===")
print(classification_report(y_test, y_pred, digits=4))
print("ROC AUC (SVM):", roc_auc_score(y_test, svm.predict_proba(X_test)[:,1]))

# Serializar y descargar el modelo
joblib.dump(svm, "svm_continuity_model.pkl")
print("✅ SVM serializado en 'svm_continuity_model.pkl'")
gfiles.download("svm_continuity_model.pkl")


=== Reporte de clasificación SVM ===
              precision    recall  f1-score   support

           0     0.9590    0.9367    0.9477       300
           1     0.9874    0.9920    0.9897      1502

    accuracy                         0.9828      1802
   macro avg     0.9732    0.9643    0.9687      1802
weighted avg     0.9827    0.9828    0.9827      1802

ROC AUC (SVM): 0.995947625388371
✅ SVM serializado en 'svm_continuity_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>